<a href="https://colab.research.google.com/github/jysung1122/aiModel/blob/main/smoke%ED%83%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#연기탐지 모델

## Pre-Trained Model 정보
# YOLO (You Only Look Once)

- 가장 빠른 객체 검출 알고리즘 중 하나
- 256x256 사이즈의 이미지
- 파이썬, 텐서플로 기반 프레임워크가 아닌 C++로 구현된 코드 기준 GPU 사용 시, 초당 170 프레임(170 FPS, frames per second)
- 작은 크기의 물체를 탐지하는데는 어려움

<img src="https://miro.medium.com/max/1400/1*bSLNlG7crv-p-m4LVYYk3Q.png" width="600">

- https://pjreddie.com/darknet/yolo/
- https://www.youtube.com/watch?v=MPU2HistivI

### 데이터셋 다운로드
* 연기 데이터셋을 방문하고 로그인해서 다운로드 가능한 URL을 받습니다.
* 연기 데이터셋: https://public.roboflow.com/object-detection/wildfire-smoke/

### YOLO모델을 다운로드 합니다.

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd /content/yolov5


In [ ]:
%mkdir smoke

폴더를 firedetect로이동해야 합니다.

In [ ]:
%cd smoke/

In [ ]:
!curl -L "https://public.roboflow.com/ds/W9zLnutya5?key=sSzT24QClK" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip


In [ ]:
from glob import glob
train_img_list = glob('/content/yolov5/smoke/train/images/*.jpg')
test_img_list = glob('/content/yolov5/smoke/test/images/*.jpg')
valid_img_list = glob('/content/yolov5/smoke/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

yaml에 데이터의 위치를 기록합니다.

In [ ]:
import yaml
with open('/content/yolov5/smoke/train.txt','w') as f:
    f.write('\n'.join(train_img_list) + '\n')
with open('/content/yolov5/smoke/test.txt','w') as f:
    f.write('\n'.join(test_img_list) + '\n')
with open('/content/yolov5/smoke/val.txt','w') as f:
    f.write('\n'.join(valid_img_list) + '\n')

In [ ]:
%cat /content/yolov5/firedetect/data.yaml

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line,cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/smoke/data.yaml

train: ./smoke/train/images
test: ./smoke/test/images
val: ./smoke/valid/images

nc: 1
names: ['smoke']

In [ ]:
%pwd

### data.yaml을 확인

In [ ]:
%cat /content/yolov5/smoke/data.yaml

### 모델구성

In [ ]:
import yaml

with open('/content/yolov5/smoke/data.yaml','r') as stream: # 이것은 순전히 num_classes를 가져오기 위함이다
  num_classes = str(yaml.safe_load(stream)['nc'])
  print("num_class = " + num_classes)
#원본의 yaml 참고용
  print("다음은 YOLO의 원본 nc - 80으로 되어 있는 파일을 보여 주고 있다. 우리는 num_classes로 바꿔야한다.")
%cat /content/yolov5/models/yolov5s.yaml

### 아래는 custom_yolo5s.yaml을 만드는 과정이다 위에서 나온 내용을 복사해서
nc: {num_classes} 라는 변수를 대입한 것만 다르다

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]

In [ ]:
%pwd

### 완성된 custom_yolov5.yaml을 확인한다.

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml


In [ ]:
%cat /content/yolov5/smoke/data.yaml

### 학습(Training)

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정 **굵은 텍스트**
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

### 아래 애러 나서 batch=16으로 바꿨음. 이거 GPU 메모리에 따라

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 16 --epochs 100 --data ./smoke/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name smoke_result --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls /content/yolov5/runs/train/smoke_result

In [ ]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
Image(filename='/content/yolov5/runs/train/smoke_result3/val_batch2_labels.jpg',width = 600)

In [ ]:
# Validation
!python val.py --weights /content/yolov5/runs/train/smoke_result/weights/best.pt --data /content/yolov5/smoke/data.yaml --img 640 --iou 0.65 --half

In [ ]:
# inference
%ls /content/yolov5/runs/train/smoke_result/weights

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/smoke_result/weights/best.pt --img 640 --conf 0.4 --source ./smoke/test/images

In [ ]:
import glob
import random
from IPython.display import Image, display

image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp/*.jpg'))

display(Image(filename=image_name))

### 모델 내보내기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%mkdir /content/drive/MyDrive/smoke
%cp /content/yolov5/runs/train/firedetect_result/weights/best.pt /content/drive/MyDrive/smoke